In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [2]:
#other imports
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [25]:
#reading in sticker type data
feature_data = sqlContext.read.parquet("gs://ds-url-catag/stick_statistics/derived_features/")


In [35]:
#basic statistics of each of the data
# feature_data.describe('sticker_packs_sent').show()

In [6]:
#convert to a vector column
from pyspark.mllib.linalg import Vectors

assembler = VectorAssembler(
    inputCols=["numofdays", "avg_con_days","sticker_packs_sent","distinct_sticker_packs_sent","sum_paid","sum_free","sum_subs","sum_discont"],
    outputCol="features")
output = assembler.transform(feature_data)
# output.select("features").show()


In [7]:
import matplotlib.pyplot as plt
# Plot the graph
plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,30):
    ax = fig.add_subplot(3,10,i+1, title='Digit: ' + str(df.loc[rndperm[i],'label']))
    ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float))


In [24]:
#computing PCA
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

pca = PCA(k=2, inputCol="normFeatures", outputCol="pcaFeatures")
model = pca.fit(l1NormData)

# df['pca-one'] = pca_result[:,0]
# df['pca-two'] = pca_result[:,1] 
# df['pca-three'] = pca_result[:,2]

In [19]:
pcamodel = model.transform(l1NormData).select("pcaFeatures")

In [ ]:
from ggplot import *
chart = ggplot( df.loc[rndperm[:3000],:], aes(x='pca-one', y='pca-two', z='pca-three', color='label') ) \
        + geom_point(size=75,alpha=0.8) \
        + ggtitle("First and Second Principal Components colored by digit")
chart

In [21]:
dataset = pcamodel.select('pcaFeatures')
dataset = dataset.withColumn('features',col('pcaFeatures'))
dataset = dataset.select('features')

In [22]:
#clustering
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(dataset)

In [23]:
#compute wssse error for those constructed model
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 222921.230165


In [13]:
# # Shows the result.
# centers = model.clusterCenters()
# print("Cluster Centers: ")
# for center in centers:
#     print(center)
# # $example off$

Cluster Centers: 
[ 0.  0.  0.  0.  0.  0.  0.  0.]
[  9.99998614e-01   0.00000000e+00   9.99998614e-01   9.99998614e-01
   0.00000000e+00   9.99997208e-01   1.40581532e-06   0.00000000e+00]
[ 0.99999861  0.          0.99999861  0.99999861  0.          0.          0.
  0.99999861]
[ 0.99999861  0.          0.99999861  0.99999861  0.99999861  0.          0.
  0.        ]


In [14]:
transformed = model.transform(dataset)

In [15]:
transformed.columns

['features', 'prediction']

In [16]:
transformed.groupby('prediction').count().show(100)

+----------+-------+
|prediction|  count|
+----------+-------+
|         1|1422660|
|         3|   1597|
|         2|  14885|
|         0|6055898|
+----------+-------+

